In [3]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd


In [4]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')
    
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')

ImportError: No module named pyspark_csv

In [ ]:
import pyspark.sql as pysql

sqlCtx = pysql.SQLContext(sc)

trip_rdd=sc.textFile('trip.csv')
trips_df =pycsv.csvToDataFrame(sqlCtx,trip_rdd)

In [ ]:
#el rdd de los trips
trips_data=trips_df.rdd

In [ ]:
#de la forma ( (start_station_id,end_station_id) , 1)  
frecuency=trips_data.map(lambda x: ((x[4],x[7]),1) ).reduceByKey(lambda x,y:x+y).collect()

In [ ]:
#pandas_df
stations=pd.read_csv('station.csv')
stations_sf=sorted(stations.loc[stations.city=='San Francisco','id'].values)
stations_sj=sorted(stations.loc[stations.city=='San Jose','id'].values)
stations_pa=sorted(stations.loc[stations.city=='Palo Alto','id'].values)
stations_rw=sorted(stations.loc[stations.city=='Redwood City','id'].values)
stations_mv=sorted(stations.loc[stations.city=='Mountain View','id'].values)


In [5]:
# frecuency_sf=data.filter(lambda x: (x[4]  in stations_sf) and(x[7]  in stations_sf) )\
#             .map(lambda x: ((x[4],x[7]),1) )\
#             .reduceByKey(lambda x,y:x+y).collect()

In [6]:
matrix_sf=[ np.zeros(len(stations_sf)) for i in range(len(stations_sf))]
matrix_sj=[ np.zeros(len(stations_sj)) for i in range(len(stations_sj))]
matrix_pa=[ np.zeros(len(stations_pa)) for i in range(len(stations_pa))]
matrix_rw=[ np.zeros(len(stations_rw)) for i in range(len(stations_rw))]
matrix_mv=[ np.zeros(len(stations_mv)) for i in range(len(stations_mv))]


NameError: name 'stations_sf' is not defined

In [7]:
def asign_value_to_matrix(city_matrix,city_stations,start_id,end_id,val):
    if (city_matrix is not None )and (city_stations is not None):
        row=city_stations.index(start_id)
        col=city_stations.index(end_id)
        city_matrix[row][col]=val

def get_trip_city(satrt_id,end_id,cities):
    for matrix,stations_list in cities:
        if (start_id in stations_list) and (end_id in stations_list):
            return matrix,stations_list
    return None,None

In [8]:
cities=[(matrix_mv,stations_mv),(matrix_pa,stations_pa),\
        (matrix_rw,stations_rw),(matrix_sf,stations_sf),\
        (matrix_sj,stations_sj)]
for (start_id,end_id),val in frecuency:
    city_matrix,city_stations = get_trip_city(start_id,end_id,cities)
    asign_value_to_matrix(city_matrix,city_stations,start_id,end_id,val)
        

NameError: name 'matrix_mv' is not defined

In [9]:
import plotly.plotly as py
import plotly.graph_objs as go


trace_sf = go.Heatmap(
    x=stations.sort_values('id').loc[stations.city=='San Francisco', ['name']].values,
    y=stations.sort_values('id').loc[stations.city=='San Francisco', ['name']].values,
    z=matrix_sf,
    autocolorscale=True
)
trace_sj = go.Heatmap(
    x=stations.sort_values('id').loc[stations.city=='San Jose', ['name']].values,
    y=stations.sort_values('id').loc[stations.city=='San Jose', ['name']].values,
    z=matrix_sj,
    autocolorscale=True
)
trace_rw = go.Heatmap(
    x=stations.sort_values('id').loc[stations.city=='Redwood City', ['name']].values,
    y=stations.sort_values('id').loc[stations.city=='Redwood City', ['name']].values,
    z=matrix_rw,
    autocolorscale=True
)
trace_pa = go.Heatmap(
    x=stations.sort_values('id').loc[stations.city=='Palo Alto', ['name']].values,
    y=stations.sort_values('id').loc[stations.city=='Palo Alto', ['name']].values,
    z=matrix_pa,
    autocolorscale=True
)
trace_mv = go.Heatmap(
    x=stations.sort_values('id').loc[stations.city=='Mountain View', ['name']].values,
    y=stations.sort_values('id').loc[stations.city=='Mountain View', ['name']].values,
    z=matrix_mv,
    autocolorscale=True
)

# layout = go.Layout(
#     xaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
#     yaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
#     autosize=True,
#     height=850,
#     width=850,
#     hovermode='closest',

# )
layout = go.Layout(
    xaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
    yaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
    autosize=True,
    height=850,
    width=850,
    hovermode='closest',
    updatemenus=list([
        dict(
            x=1,
            y=1.06,
            buttons=list([
                dict(
                    args=['visible', [True, False, False, False,False]],
                    label='San Francisco',
                    
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, True, False, False,False]],
                    label='San Jose',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, True, False,False]],
                    label='Palo Alto',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, True,False]],
                    label='Mountain View',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False, False, False, False,True]],
                    label='Redwood City',
                    method='restyle'
                )
            ]),
            yanchor='top'
        )
    ]),
)




data = [trace_sf,trace_sj,trace_pa,trace_mv,trace_rw]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig,filename='datetime-heatmap')


ImportError: No module named plotly.plotly